In [509]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt

<h2>Finding a real estate agent based on location of the property</h2>
<p><b>This substudy was carried out as an auxiliary recommender system to our main solution, but was not included in the report.</b>The purpose of this substudy is to find which real estate agent would be most reliable for selling the property. When we tried to solve this problem with deep NN, we got 0.28 classification accuracy on the test data set — such a low score might be connected with the fact that a large number of agents in the test set were not present in the training set, hence could not be possibly predicted. However, we think that the underlying principle of selecting agents can be different with respect to how flexibly the agents can be selected. Further we propose an appoach based on the Association Rule Mining. Our method will be based on the location the house, and will search for agents which have the best deal sealing rate.</p>

<h2>Data preparation</h2>

In [510]:
data = pd.read_csv('./Dataset/House_Price_Dataset.csv')
data.head()

,Suburb,Address,Rooms,Type,Price,Method,Seller,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Latitude,Longitude,RegionName,PropertyCount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,03/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,03/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,04/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,04/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,04/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [511]:
# CouncilArea had a big cumulative importance when predicting selling price with the Random Forest, so we decided to utilize
# it as an input variable here
data = data[~(data['CouncilArea'].isnull())]
data = data[~(data['Seller'].isnull())]
data = data[~(data['Method'].isnull())]

In [512]:
council_areas = np.unique(data["CouncilArea"])

In [513]:
council_areas

array(['Banyule City Council', 'Bayside City Council',
       'Boroondara City Council', 'Brimbank City Council',
       'Cardinia Shire Council', 'Casey City Council',
       'Darebin City Council', 'Frankston City Council',
       'Glen Eira City Council', 'Greater Dandenong City Council',
       'Hobsons Bay City Council', 'Hume City Council',
       'Kingston City Council', 'Knox City Council',
       'Macedon Ranges Shire Council', 'Manningham City Council',
       'Maribyrnong City Council', 'Maroondah City Council',
       'Melbourne City Council', 'Melton City Council',
       'Mitchell Shire Council', 'Monash City Council',
       'Moonee Valley City Council', 'Moorabool Shire Council',
       'Moreland City Council', 'Nillumbik Shire Council',
       'Port Phillip City Council', 'Stonnington City Council',
       'Whitehorse City Council', 'Whittlesea City Council',
       'Wyndham City Council', 'Yarra City Council',
       'Yarra Ranges Shire Council'], dtype=object)

In [514]:
np.unique(data["Seller"]).size

388

In [515]:
seller_gr = data.groupby(['Seller']).size().sort_values(ascending=False).reset_index(name='counts')
seller_gr.head()

,Seller,counts
0,Jellis,3358
1,Nelson,3236
2,Barry,3235
3,hockingstuart,2623
4,Marshall,2027


In [516]:
field = "CouncilArea"
cut = data[["Method", "Seller", field]]
cut = cut[(~cut['Method'].isnull()) & (~cut['Seller'].isnull()) & (~cut[field].isnull())]
cut.head()

,Method,Seller,CouncilArea
0,SS,Jellis,Yarra City Council
1,S,Biggin,Yarra City Council
2,S,Biggin,Yarra City Council
3,VB,Rounds,Yarra City Council
4,SP,Biggin,Yarra City Council


In [517]:
print(np.unique(cut["Method"]))

['PI' 'PN' 'S' 'SA' 'SN' 'SP' 'SS' 'VB' 'W']


In [518]:
# encoding whether the house was sold or not sold
cut["Method"] = cut["Method"].apply(lambda x: 'S' if x.startswith('S') else 'N')

In [519]:
methods = [frozenset(x) for x in np.unique(cut["Method"])]
methods

[frozenset({'N'}), frozenset({'S'})]

In [520]:
np_cut = cut.to_numpy()

In [521]:
# encoding for frequent pattern finding
te = TransactionEncoder()
te_ary = te.fit(np_cut).transform(np_cut)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.head()

,@Realty,A,AIME,ASL,Abercromby's,Ace,Airport,Alex,Alexkarbon,Allan,...,hockingstuart/Marshall,hockingstuart/Sweeney,hockingstuart/Village,hockingstuart/hockingstuart,iHomes,iOne,iProperty,iSell,iTRAK,voglwalpole
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Experiments (for illustration only)

In [490]:
res = fpgrowth(df, min_support=1/10**7, use_colnames=True, max_len=3)

In [491]:
res.head()

,support,itemsets
0,0.757904,(S)
1,0.096345,(Jellis)
2,0.034028,(Yarra City Council)
3,0.025736,(Biggin)
4,0.242096,(N)


In [492]:
res[res['itemsets'].map(len) == 3].sort_values(by=['support'], ascending=False)

,support,itemsets
427,0.021777,"(S, Boroondara City Council, Jellis)"
786,0.020285,"(Marshall, Boroondara City Council, S)"
1185,0.017100,"(Nelson, S, Darebin City Council)"
2624,0.014604,"(Nelson, S, Moreland City Council)"
802,0.014059,"(Nelson, S, Moonee Valley City Council)"
...,...,...
2824,0.000029,"(S, Property, Glen Eira City Council)"
2817,0.000029,"(Moreland City Council, N, Kelly)"
2816,0.000029,"(Boroondara City Council, N, Kelly)"
2815,0.000029,"(S, Manningham City Council, Kelly)"


In [493]:
ar = association_rules(res, metric="support", min_threshold=1/10**7)

In [494]:
ar.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(S),(Jellis),0.757904,0.096345,0.071269,0.094034,0.976015,-0.001751,0.997449
1,(Jellis),(S),0.096345,0.757904,0.071269,0.739726,0.976015,-0.001751,0.930157
2,(Jellis),(N),0.096345,0.242096,0.025076,0.260274,1.075088,0.001751,1.024574
3,(N),(Jellis),0.242096,0.096345,0.025076,0.103579,1.075088,0.001751,1.008070
4,(Boroondara City Council),(Jellis),0.105440,0.096345,0.030327,0.287619,2.985311,0.020168,1.268500


In [495]:
ar = ar[(ar['antecedents'].map(len) == 2) 
       & ((ar['consequents'].map(len) == 1) 
            & (ar['consequents'].isin(methods)))
  ].sort_values(by=['antecedent support', 'lift'], ascending=False)

In [496]:
ar.head(100)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
7,"(Boroondara City Council, Jellis)",(N),0.030327,0.242096,0.008550,0.281930,1.164540,0.001208,1.055474
14,"(Boroondara City Council, Jellis)",(S),0.030327,0.757904,0.021777,0.718070,0.947441,-0.001208,0.858708
1562,"(Marshall, Boroondara City Council)",(S),0.026740,0.757904,0.020285,0.758584,1.000896,0.000018,1.002814
1556,"(Marshall, Boroondara City Council)",(N),0.026740,0.242096,0.006456,0.241416,0.997194,-0.000018,0.999105
3257,"(Nelson, Darebin City Council)",(S),0.021662,0.757904,0.017100,0.789404,1.041561,0.000682,1.149573
...,...,...,...,...,...,...,...,...,...
6350,"(HAR, Whittlesea City Council)",(N),0.005566,0.242096,0.001090,0.195876,0.809087,-0.000257,0.942522
3327,"(Darebin City Council, McGrath)",(N),0.005451,0.242096,0.001348,0.247368,1.021780,0.000029,1.007006
3321,"(Darebin City Council, McGrath)",(S),0.005451,0.757904,0.004103,0.752632,0.993043,-0.000029,0.978684
11787,"(Nelson, Brimbank City Council)",(S),0.005021,0.757904,0.004074,0.811429,1.070621,0.000269,1.283840


In [ ]:
def find_opposite_support(df, antecedents):
    item = df[(df['antecedents'] == antecedents) & (df['consequents_unpacked'] == 0)]
    if not item.empty: 
        return item.iloc[0].support
    else: 
        return 1

In [497]:
ar = ar[ar['antecedents'].map({'Boroondara City Council'}.intersection).astype(bool)]
ar['consequents_unpacked'] = (ar.apply(lambda x: 0 if ", ".join(list(x.consequents)) == "N" else 1, axis=1))
ar['ratio'] = (ar.apply(lambda x: 0 if x.consequents_unpacked == "N" else x.support / find_opposite_support(ar, x.antecedents), axis=1))
ar = ar[~(ar['consequents_unpacked'] == 0)].sort_values(['ratio'], ascending=[False])

In [498]:
ar.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,consequents_unpacked,ratio
8966,"(Boroondara City Council, J)",(S),0.000603,0.757904,0.000574,0.952381,1.256598,0.000117,5.084008,1,20.0
10106,"(Boroondara City Council, Abercromby's)",(S),0.000287,0.757904,0.000258,0.900000,1.187485,0.000041,2.420956,1,9.0
11320,"(Boroondara City Council, Domain)",(S),0.000258,0.757904,0.000230,0.888889,1.172825,0.000034,2.178860,1,8.0
1122,"(Philip, Boroondara City Council)",(S),0.000201,0.757904,0.000172,0.857143,1.130938,0.000020,1.694669,1,6.0
381,"(Nelson, Boroondara City Council)",(S),0.003213,0.757904,0.002640,0.821429,1.083816,0.000204,1.355735,1,4.6


## Functional Implementation

<p>This is a neat implementation of our concept</p>

In [527]:
def find_opposite_support(df, antecedents):
    item = df[(df['antecedents'] == antecedents) & (df['consequents_unpacked'] == 0)]
    if not item.empty: 
        return item.iloc[0].support
    else: 
        return 1

<p>We apply two metrics interest and support ratio (sold/not sold). They can show us which real estate agent have a higher successful deals rate. The experience of the agent can be tuned by adjusting support level.</p>

In [528]:
def select_seller(data, council_area, metric, min_support_association_rule=1/10**7):
    st = frozenset({council_area})
    res = fpgrowth(data, min_support=1/10**7, use_colnames=True, max_len=3)
    res[res['itemsets'].map(len) == 3].sort_values(by=['support'], ascending=False)
    ar = association_rules(res, metric="support", min_threshold=min_support_association_rule)
    ar = ar[(ar['antecedents'].map(len) == 2) 
       & ((ar['consequents'].map(len) == 1) 
            & (ar['consequents'].isin(methods)))
    ].sort_values(by=['antecedent support', 'lift'], ascending=False)
    ar['consequents_unpacked'] = (ar.apply(lambda x: 0 if ", ".join(list(x.consequents)) == "N" else 1, axis=1))
    ar = ar[ar['antecedents'].map(st.intersection).astype(bool)]
    if metric == 'ratio':
        ar['ratio'] = (ar.apply(lambda x: 0 if x.consequents_unpacked == "N" else x.support / find_opposite_support(ar, x.antecedents), axis=1))
        ar = ar[~(ar['consequents_unpacked'] == 0)].sort_values(['ratio'], ascending=[False])
    elif metric == 'interest':
        ar = ar[(ar['consequents_unpacked'] == 1)].sort_values(['lift'], ascending=[False])
    else: 
        return ''
    if not ar.empty:
        an = ar.iloc[0].antecedents
        res = an.difference(st)
        if len(res) == 0:
            return ''
        else:
            return list(res)[0]
    else:
        return ''

In [529]:
ratio = []
for x in council_areas:
    ratio.append(select_seller(df, x, 'ratio'))

In [530]:
interest = []
for x in council_areas:
    interest.append(select_seller(df, x, 'interest', 0.001))

In [531]:
pd.DataFrame(list(zip(ratio, interest)), columns=['support_ratio_agent', 'interest_agent'])

,support_ratio_agent,interest_agent
0,William,Buckingham
1,Thomson,Nick
2,J,Nelson
3,Ray,Bells
4,Ray,
5,Harcourts,O'Brien
6,Biggin,Brad
7,Harcourts,O'Brien
8,C21,Woodards
9,Del,Barry
